# Join and Score Model M043

In [44]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_absolute_error
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
def each_group_score(oof):
    for t, d in oof.groupby('type'):
        type_mae = mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds'])
        print('{} - MAE {:0.4f} - LOG MAE {:0.4f}'.format(t,
                                                          type_mae,
                                                         np.log(type_mae)))
    gmlm_score = group_mean_log_mae(oof['scalar_coupling_constant'],
                                    oof['oof_preds'],
                                    oof['type'])
    print('ALL MAE {:0.4f}'.format(mean_absolute_error(oof['scalar_coupling_constant'],
                                                      oof['oof_preds'])))
    print('ALL GROUP LOG MAE {:0.4f}'.format(gmlm_score))

In [18]:
MODEL_NO = 'M043'
oof_type_file = [x for x in os.listdir(f'../type_results/{t}') if MODEL_NO in x and 'oof' in x][-1]
sub_type_file = [x for x in os.listdir(f'../type_results/{t}') if MODEL_NO in x and 'sub' in x][-1]

In [30]:
types = ['2JHN', '3JHN', '2JHC', '3JHC', '2JHH', '1JHN', '3JHH', '1JHC']
oof = pd.DataFrame()
sub = pd.DataFrame()
for t in types:
    oof_type_file = [x for x in os.listdir(f'../type_results/{t}') if MODEL_NO in x and 'oof' in x][-1]
    sub_type_file = [x for x in os.listdir(f'../type_results/{t}') if MODEL_NO in x and 'sub' in x][-1]
    oof_type = pd.read_parquet(f'../type_results/{t}/{oof_type_file}')
    sub_type = pd.read_parquet(f'../type_results/{t}/{sub_type_file}')
    sub_type = sub_type.loc[sub_type['type'] == t] # Filter to correct type
    
    sub = pd.concat([sub, sub_type])
    oof = pd.concat([oof, oof_type])

In [36]:
sub.sort_values('id').head()

,id,type,molecule_name,prediction
0,4658147,2JHC,dsgdb9nsd_000004,15.570594
1,4658148,1JHC,dsgdb9nsd_000004,180.471621
2,4658149,3JHH,dsgdb9nsd_000004,6.703215
3,4658150,1JHC,dsgdb9nsd_000004,179.013627
4,4658151,2JHC,dsgdb9nsd_000004,15.137757


In [45]:
each_group_score(oof)

1JHC - MAE 0.5124 - LOG MAE -0.6686
1JHN - MAE 0.3764 - LOG MAE -0.9771
2JHC - MAE 0.2137 - LOG MAE -1.5432
2JHH - MAE 0.1382 - LOG MAE -1.9791
2JHN - MAE 0.1478 - LOG MAE -1.9119
3JHC - MAE 0.2462 - LOG MAE -1.4016
3JHH - MAE 0.1552 - LOG MAE -1.8630
3JHN - MAE 0.1199 - LOG MAE -2.1209
ALL MAE 0.2527
ALL GROUP LOG MAE -1.5582


In [37]:
oof.sort_values('id').head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,87.367459
1,1,2JHH,-11.2570,-10.350698
2,2,2JHH,-11.2548,-10.631398
3,3,2JHH,-11.2543,-10.805525
4,4,1JHC,84.8074,87.665804
